In [1]:
import pandas as pd
import requests

In [32]:
df = pd.read_csv('data.csv')
df['published_at'] = (pd.to_datetime(df['published_at'])).dt.date
df.head()

,id,name,schedule,accept_temporary,published_at,city,salary_mean
0,74376536,Data scientist,remote,False,2022-12-29,Москва,175000.0
1,74157498,Data scientist,fullDay,False,2022-12-22,Москва,170000.0
2,54372021,TechLead Research Engineer (NLP/Speech),remote,False,2022-12-23,Москва,350000.0
3,54372020,TechLead Research Engineer (NLP/Speech),remote,False,2022-12-23,Санкт-Петербург,350000.0
4,54372019,TechLead Research Engineer (NLP/Speech),remote,False,2022-12-23,Иннополис,350000.0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33 entries, 0 to 32
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                33 non-null     int64  
 1   name              33 non-null     object 
 2   schedule          33 non-null     object 
 3   accept_temporary  33 non-null     bool   
 4   published_at      33 non-null     object 
 5   city              33 non-null     object 
 6   salary_mean       33 non-null     float64
dtypes: bool(1), float64(1), int64(1), object(4)
memory usage: 1.7+ KB
